In [58]:
import pandas as pd
from tqdm.auto import tqdm
import actions
from pathlib import Path
from utils import *
import re

In [4]:
# Define the file paths
parquet_file_abcd = Path("../data/ABCD_tripfiles.parquet")
parquet_file_abcd_conv = Path("../data/ABCD_tripfiles_conv.parquet")
parquet_file_mnop = Path("../data/MNOP_tripfiles.parquet")
parquet_file_mnop_conv = Path("../data/MNOP_tripfiles_conv.parquet")
parquet_file_zyxw = Path("../data/ZYXW_tripfiles.parquet")
parquet_file_zyxw_conv = Path("../data/ZYXW_tripfiles_conv.parquet")
print(
    parquet_file_abcd,
    parquet_file_abcd_conv,
    parquet_file_mnop,
    parquet_file_mnop_conv,
    parquet_file_zyxw,
    parquet_file_zyxw_conv,
)

..\data\ABCD_tripfiles.parquet ..\data\ABCD_tripfiles_conv.parquet ..\data\MNOP_tripfiles.parquet ..\data\MNOP_tripfiles_conv.parquet ..\data\ZYXW_tripfiles.parquet ..\data\ZYXW_tripfiles_conv.parquet


In [138]:
df_abcd = pd.read_parquet(parquet_file_abcd)
df_mnop = pd.read_parquet(parquet_file_mnop)
df_zyxw = pd.read_parquet(parquet_file_zyxw)

In [6]:
print(round(df_abcd.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
print(round(df_mnop.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
print(round(df_zyxw.memory_usage(deep=True).sum() / 1024**2, 2), "MB")

1214.94 MB
2219.62 MB
585.54 MB


In [7]:
action_names = set(df_abcd.action_name.unique())
len(action_names)

49

In [8]:
action_names.update(set(df_mnop.action_name.unique()))
action_names.update(set(df_zyxw.action_name.unique()))


In [9]:
action_extractors = {
    "CalculateWeightAndTrimAction": actions.CalculateWeightAndTrimAction.extract,
    "CheckinMsgProcessor": None,
    "CreateLoadsheetAction": None,
    "CreateZFWMessageAction": None,
    "SendFuelOrderAction": None,
    "SendLoadsheetAction": None,
    "SetActualBagWeightIndicatorAction": None,
    "StorePaxDataAction": None,
    "StoreRegistrationAndConfigurationAc": None,
    "UpdateFuelDataAction": None,
    "RampFinalAction": actions.RampFinalAction.extract,
    "CreateLoadingInstructionAction": None,
    "CopyPaxDataAction": None,
    "CreateBaggageLoadItemsAction": None,
    "EstimateStorePaxDataAction": None,
    "SendLoadingInstructionAction": None,
    "SetCKIPaxDistributionAction": None,
    "SpecialPaxWeightAction": None,
    "StoreAircraftDataAction": None,
    "StoreCKIAverageWeightAction": None,
    "StorePaxDataGuiAction": None,
    "TdmCreateLoadingInstructionAction": None,
    "TransferCargoAction": None,
    "TransferCheckinDataAction": None,
    "UpdateCargoMailEstimatesAction": None,
    "UpdateEstimatesAction": None,
    "UpdateLastEzfwSentAction": None,
    "UpdateLoadTableAction": None,
}

AttributeError: module 'actions.CalculateWeightAndTrimAction' has no attribute 'test'

In [125]:

tqdm.pandas()

In [55]:
for action_name, extractor in action_extractors.items():
    tqdm.pandas()
    if extractor is not None:
        df_abcd[f"data_{action_name}"] = df_abcd[df_abcd.action_name == action_name][
            "entry_details"
        ].progress_apply(extractor)

df_abcd.to_parquet(parquet_file_abcd_conv, engine="pyarrow", compression="brotli")

100%|██████████| 231990/231990 [00:13<00:00, 17790.56it/s]


In [56]:
# df = pd.read_parquet(parquet_file_abcd_conv)

In [107]:
df = df_zyxw
df[df.action_name == "RampFinalAction"]["entry_details"].unique()
# RampFinalAction
entry_details_contains = "LOADING_INSTRUCTION"
entry_details_contains = "LOADSHEET"
"Caller user"

x = df[
    (df["action_name"] == "RampFinalAction")
    & (~df["entry_details"].str.contains("LOZYING_INSTRUCTION ", na=False))
    & (~df["entry_details"].str.contains("LOADSHEET", na=False))
    # & (~df["entry_details"].str.contains("Discrepancy check result", na=False))
    & (~df["entry_details"].str.contains("Caller user", na=False))  # don't include this
]["entry_details"].unique()


# STATUS LOADING_INSTRUCTION 1 FUEL 4 AIRCRAFT_CONFIG 1 EZFW 3 CARGO_FINAL 1 RAMP_FINAL 1 CARGO_TRANSFER 1 CABIN_CONFIG 1 CALC_HIST_DATA 1 CHECK_IN_OPEN 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 3 REGISTRATION 1 REGISTRATION_CHANGE 5'

In [ ]:
foo: set = set()

In [103]:
for message in x:
    items = message.split(" ")
    for item in items:
        if not item.isdigit():
            foo.add(item)

In [104]:
foo

{'AIRBORNE',
 'AIRCRAFT_CONFIG',
 'ALLOWANCE_CHECK_PERFORMED',
 'AUTOMATION_STARTED',
 'AUTO_MODE_ACTIVE',
 'BAG_LOAD_ITEMS_GEN',
 'BAG_LOZY_ITEMS_GEN',
 'BAG_ULD_ORD',
 'CABIN_CONFIG',
 'CALC_HIST_DATA',
 'CARGO_FINAL',
 'CARGO_TRANSFER',
 'CHECK_IN_FINAL',
 'CHECK_IN_OPEN',
 'DGR_ITEMS',
 'EZFW',
 'EZFW_COUNTER',
 'FINAL_RELEASE',
 'FUEL',
 'FUEL_ORDER',
 'LDM',
 'LOADING_INSTRUCTION',
 'LOADSHEET',
 'LOZYING_INSTRUCTION',
 'LOZYSHEET',
 'OFFBLOCK',
 'OFP',
 'PDM',
 'RAMP_FINAL',
 'REGISTRATION',
 'REGISTRATION_CHANGE',
 'STATUS',
 'TRANSIT_ACCEPTANCE',
 'TRANSIT_PAX'}

In [139]:
x = df_abcd.copy()
x["data_RampFinalAction"] = df_abcd[df_abcd.action_name == "RampFinalAction"][
    "entry_details"
].progress_apply(extract_key_value_pairs)

100%|██████████| 3804/3804 [00:00<00:00, 50359.13it/s]


In [147]:
def extract_key_value_pairs(message: str):
    typos = {
        "BAG_LOZY_ITEMS_GEN": "BAG_LOAD_ITEMS_GEN",
        "LOZYING_INSTRUCTION": "LOADING_INSTRUCTION",
        "LOZYSHEET": "LOADSHEET",
    }  # Fix typos in the messages from zyxw
    for key, value in typos.items():
        message = message.replace(key, value)

    if "STATUS" in message:
        keys = [
            "AIRBORNE",
            "AIRCRAFT_CONFIG",
            "ALLOWANCE_CHECK_PERFORMED",
            "AUTOMATION_STARTED",
            "AUTO_MODE_ACTIVE",
            "BAG_LOAD_ITEMS_GEN",
            "BAG_LOZY_ITEMS_GEN",
            "BAG_ULD_ORD",
            "CABIN_CONFIG",
            "CALC_HIST_DATA",
            "CARGO_FINAL",
            "CARGO_TRANSFER",
            "CHECK_IN_FINAL",
            "CHECK_IN_OPEN",
            "DGR_ITEMS",
            "EZFW",
            "EZFW_COUNTER",
            "FINAL_RELEASE",
            "FUEL",
            "FUEL_ORDER",
            "LDM",
            "LOADING_INSTRUCTION",
            "LOADSHEET",
            "LOZYING_INSTRUCTION",
            "LOZYSHEET",
            "OFFBLOCK",
            "OFP",
            "PDM",
            "RAMP_FINAL",
            "REGISTRATION",
            "REGISTRATION_CHANGE",
            "TRANSIT_ACCEPTANCE",
            "TRANSIT_PAX",
        ]

        # Initialize dictionary with None for all keys
        data = {key: None for key in keys}

        # Split the message into parts
        parts = message.split()

        # Iterate through parts and extract key-value pairs
        i = 0
        while i < len(parts):
            if parts[i] in keys:
                key = parts[i]
                if i + 1 < len(parts) and re.match(r"^-?\d+(\.\d+)?$", parts[i + 1]):
                    value = parts[i + 1]
                    data[key] = int(value)
                    i += 2
                else:
                    i += 1
            else:
                i += 1

        return data
    if "Discrepancy check result" in message:
        # Initialize an empty dictionary
        result = {"Discrepancy check result": {}, "Discrepancies": []}

        # Split the report into lines
        lines = message.split("\n")
        # print(lines)

        # Parse the first part of the report
        result["Discrepancy check result"]["Discrepancy happened"] = (
            lines[1].split(": ")[1].strip() == "true"
        )

        # Parse the discrepancies
        for line in lines[
            4:7
        ]:  # One error in the data where the loadtable is repeated twice
            print(line)
            if line.strip():
                columns = line.split()
                discrepancy = {
                    "Type": columns[0],
                    "Destination": columns[1],
                    "Bag pieces": int(columns[2]),
                    "Bag weight": float(columns[3].replace("KG", "").strip()),
                }
                result["Discrepancies"].append(discrepancy)

        return result
    if "Caller user" in message:
        return None
    raise ValueError(f"Unknown message: {message}")

SyntaxError: unmatched ')' (814396953.py, line 1)

In [120]:
extract_key_value_pairs(x[2])

LOZYTABLE                  REC                       298                       5960.00 KG                
CKI                        REC                       0                         0.00    KG                
SUM                        REC                       298                       5960.00 KG                


{'Discrepancy check result': {'Discrepancy happened': True},
 'Discrepancies': [{'Type': 'LOADTABLE',
   'Destination': 'REC',
   'Bag pieces': 298,
   'Bag weight': 5960.0},
  {'Type': 'CKI', 'Destination': 'REC', 'Bag pieces': 0, 'Bag weight': 0.0},
  {'Type': 'SUM',
   'Destination': 'REC',
   'Bag pieces': 298,
   'Bag weight': 5960.0}]}

In [47]:
x

array(['STATUS LOADING_INSTRUCTION 1 FUEL 4 AIRCRAFT_CONFIG 1 EZFW 3 CARGO_FINAL 1 RAMP_FINAL 1 CARGO_TRANSFER 1 CABIN_CONFIG 1 CALC_HIST_DATA 1 CHECK_IN_OPEN 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 3 REGISTRATION 1 REGISTRATION_CHANGE 5',
       'STATUS LOADING_INSTRUCTION 1 FUEL 3 AIRCRAFT_CONFIG 1 EZFW 2 CARGO_FINAL 1 RAMP_FINAL 1 CARGO_TRANSFER 1 CABIN_CONFIG 1 CALC_HIST_DATA 1 CHECK_IN_OPEN 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 2 REGISTRATION 1 REGISTRATION_CHANGE 3',
       'STATUS LOADING_INSTRUCTION 2 FUEL 3 AIRCRAFT_CONFIG 1 EZFW 2 CARGO_FINAL 1 RAMP_FINAL 1 CARGO_TRANSFER 1 CABIN_CONFIG 1 CALC_HIST_DATA 1 CHECK_IN_OPEN 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 2 REGISTRATION 1 REGISTRATION_CHANGE 6',
       'STATUS LOADING_INSTRUCTION 2 FUEL 2 AIRCRAFT_CONFIG 1 EZFW 2 CARGO_FINAL 1 RAMP_FINAL 1 CARGO_TRANSFER 1 CABIN_CONFIG 1 TRANSIT_ACCEPTANCE 1 TRANSIT_PAX 1 CALC_HIST